# 5.1 层和块
块（block），层（layer）
*从编程的角度来看，块由类（class）表示。它的任何子类都必须定义一个将其输入转换为输出的前向传播函
数，并且必须存储任何必需的参数。在定义我们自己的块时，一般我们只需要考虑前向传播函数和必要的参数*

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(), 
                    nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)
# 这里我们通过实例化Sequential来进行构建模型，Sequential是一种特殊的Module也就是一种快

tensor([[-0.1541, -0.1779,  0.0860, -0.0023,  0.2044, -0.0051, -0.0590,  0.0081,
          0.1588, -0.1864],
        [-0.0716, -0.1415,  0.0373, -0.1479,  0.1410, -0.1132,  0.1019, -0.0542,
          0.2021, -0.0589]], grad_fn=<AddmmBackward0>)

## 5.1.1 自定义块
每一个块必须提供的基本功能：
1. 将输入数据作为其前向传播函数的参数
2. 通过前向传播函数来生成输出。输出与输入的形状不必相同
3. 计算其输出关于输入的梯度，可以通过其反向传播函数进行访问*这常常时自动发生的*
4. 存储和访问前向传播计算所需要的参数
5. 根据需要初始化模型参数


In [3]:
class MLP(nn.Module):
    # 用参数声明层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层

    # 定义模型的前向传播
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[ 2.2541e-01,  1.3199e-01,  2.4315e-04,  6.1485e-02, -6.4898e-02,
         -1.6717e-01,  8.3730e-02, -3.2662e-03, -2.3478e-01,  2.3014e-01],
        [ 2.3603e-01,  5.3410e-03,  5.3815e-02,  3.8420e-02, -1.1678e-01,
         -2.0767e-01,  5.0509e-02, -3.5736e-03, -3.7410e-01,  1.2011e-01]],
       grad_fn=<AddmmBackward0>)

In [5]:
# 块具有比较好的结构性，类似于逐层的抽象

In [7]:
# 顺序块 Sequential类
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0357, -0.0949,  0.2539, -0.1732,  0.1989, -0.0606, -0.1905, -0.1554,
          0.1482,  0.0725],
        [ 0.1042, -0.0578,  0.2320, -0.0969,  0.2381, -0.0446, -0.0996, -0.1626,
         -0.0292,  0.2149]], grad_fn=<AddmmBackward0>)

## 5.1.3 在前向传播函数中执行代码
Sequential 使得模型构造变得简单，允许我们组合新的架构**很多时候我们需要更强的灵活性，定义自己的块**，以便于在前向传播中**利用控制流**，或者是使用其他数学运算
目前为止，我们网络中的操作对网络的激活值以及网络参数起作用，但是，有时我们希望更新*函数参数*

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [12]:
net = FixedHiddenMLP()
net(X)

tensor(0.4425, grad_fn=<SumBackward0>)